<a href="https://colab.research.google.com/github/dolphin1404/AI_lab/blob/main/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 도서-스크립트 변환 프로젝트: 데이터 수집 및 전처리

## 프로젝트 개요
- **목표**: 도서 텍스트를 비디오 스크립트 형식으로 변환하는 LLM 모델 개발
- **단계**: 데이터 수집 → 전처리 → 모델링 → 성능평가
- **일정**: 데이터 전처리 마감 - 10월 27일

## 데이터 전처리 목표
1. 공개 도서 아카이브에서 텍스트 데이터 수집 (Project Gutenberg, 국내 디지털 도서관)
2. 노이즈 제거 및 텍스트 정제
3. 핵심 요소 추출 (인물, 장소, 시간)
4. 도서 문체 → 스크립트 문체 변환을 위한 학습 데이터셋 구축

## 1. 환경 설정 및 라이브러리 설치

In [ ]:
# 필요한 라이브러리 설치
!pip install gutenberg requests beautifulsoup4 nltk spacy transformers datasets
!python -m spacy download en_core_web_sm
!python -m spacy download ko_core_news_sm

In [ ]:
# 라이브러리 임포트
import os
import re
import requests
import nltk
import spacy
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from collections import defaultdict
import json

# NLTK 데이터 다운로드
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

# SpaCy 모델 로드
nlp_en = spacy.load('en_core_web_sm')

print("✓ All libraries imported successfully!")

## 2. 데이터 수집 (Data Collection)

### 2.1 Project Gutenberg에서 도서 다운로드

In [ ]:
class GutenbergCollector:
    """Project Gutenberg에서 도서 데이터를 수집하는 클래스"""
    
    def __init__(self, base_url="https://www.gutenberg.org"):
        self.base_url = base_url
        
    def download_book(self, book_id):
        """
        특정 책 ID로 도서 텍스트 다운로드
        
        Args:
            book_id (int): Gutenberg 도서 ID
        
        Returns:
            str: 도서 텍스트 내용
        """
        url = f"{self.base_url}/files/{book_id}/{book_id}-0.txt"
        try:
            response = requests.get(url, timeout=10)
            if response.status_code == 200:
                return response.text
            else:
                print(f"Failed to download book {book_id}: Status {response.status_code}")
                return None
        except Exception as e:
            print(f"Error downloading book {book_id}: {str(e)}")
            return None
    
    def get_popular_books(self, genre="fiction", limit=10):
        """
        장르별 인기 도서 ID 리스트 반환
        
        Args:
            genre (str): 도서 장르
            limit (int): 다운로드할 최대 도서 수
        
        Returns:
            list: 도서 ID 리스트
        """
        # 인기 고전 소설 ID 리스트 (예시)
        popular_fiction = [
            1342,  # Pride and Prejudice by Jane Austen
            84,    # Frankenstein by Mary Shelley
            98,    # A Tale of Two Cities by Charles Dickens
            1661,  # Sherlock Holmes by Arthur Conan Doyle
            2701,  # Moby Dick by Herman Melville
            11,    # Alice's Adventures in Wonderland by Lewis Carroll
            74,    # The Adventures of Tom Sawyer by Mark Twain
            1260,  # Jane Eyre by Charlotte Brontë
            844,   # The Importance of Being Earnest by Oscar Wilde
            2852   # The Hound of the Baskervilles by Arthur Conan Doyle
        ]
        return popular_fiction[:limit]

# 사용 예시
collector = GutenbergCollector()
print("✓ Gutenberg Collector initialized")
print(f"Sample book IDs: {collector.get_popular_books(limit=5)}")

In [ ]:
# 샘플 도서 다운로드
sample_book_id = 1342  # Pride and Prejudice
book_text = collector.download_book(sample_book_id)

if book_text:
    print(f"✓ Successfully downloaded book {sample_book_id}")
    print(f"Book length: {len(book_text)} characters")
    print("\nFirst 500 characters:")
    print(book_text[:500])
else:
    print("Failed to download sample book")

## 3. 데이터 전처리 (Data Preprocessing)

### 3.1 텍스트 정제 (Text Cleaning)

In [ ]:
class TextPreprocessor:    """텍스트 전처리를 위한 클래스"""        def __init__(self):        self.nlp = nlp_en        def remove_gutenberg_header_footer(self, text):        """        Project Gutenberg 헤더와 푸터 제거                Args:            text (str): 원본 텍스트                Returns:            str: 헤더/푸터가 제거된 텍스트        """        # 시작 마커 찾기        start_markers = [            "*** START OF THIS PROJECT GUTENBERG",            "*** START OF THE PROJECT GUTENBERG",            "***START OF THE PROJECT GUTENBERG"        ]                # 종료 마커 찾기        end_markers = [            "*** END OF THIS PROJECT GUTENBERG",            "*** END OF THE PROJECT GUTENBERG",            "***END OF THE PROJECT GUTENBERG"        ]                start_idx = 0        for marker in start_markers:            idx = text.find(marker)            if idx != -1:                start_idx = text.find('\n', idx) + 1                break                end_idx = len(text)        for marker in end_markers:            idx = text.find(marker)            if idx != -1:                end_idx = idx                break                return text[start_idx:end_idx].strip()        def clean_text(self, text):        """        기본 텍스트 정제        - 과도한 공백 제거        - 특수 문자 정규화        - 줄바꿈 정규화                Args:            text (str): 원본 텍스트                Returns:            str: 정제된 텍스트        """        # 여러 줄바꿈을 단일 줄바꿈으로        text = re.sub(r'\n\s*\n', '\n\n', text)                # 여러 공백을 단일 공백으로        text = re.sub(r' +', ' ', text)                # 줄 시작/끝 공백 제거        text = '\n'.join(line.strip() for line in text.split('\n'))                return text.strip()        def split_into_chapters(self, text):        """        텍스트를 챕터별로 분할 (개선된 버전)                다양한 챕터 형식 지원:        - "CHAPTER I", "CHAPTER 1", "CHAPTER ONE"        - "Chapter I.", "Chapter 1.", "Chapter One."        - "I.", "1.", "ONE."        - "I\n", "1\n" (숫자/로마자만 있는 줄)                Args:            text (str): 전체 텍스트                Returns:            list: 챕터별 텍스트 리스트        """        # 다양한 챕터 패턴을 시도        patterns = [            # Pattern 1: "CHAPTER I" or "Chapter 1" with optional period/colon            r'\n\s*(CHAPTER|Chapter)\s+([IVXLCDM]+|\d+|One|Two|Three|Four|Five|Six|Seven|Eight|Nine|Ten)[\s\.:\n]',            # Pattern 2: Roman numerals or numbers at start of line (standalone)            r'\n\s*([IVXLCDM]+|\d+)\.?\s*\n',            # Pattern 3: "BOOK I", "PART I", etc.            r'\n\s*(BOOK|Book|PART|Part)\s+([IVXLCDM]+|\d+)[\s\.:\n]',        ]                best_split = None        best_count = 0                # Try each pattern and use the one that gives the most reasonable splits        for pattern in patterns:            splits = re.split(pattern, text)            # Count potential chapters (excluding first element which is pre-chapter content)            chapter_count = (len(splits) - 1) // 3 if len(splits) > 1 else 0                        # Prefer pattern that splits into 3-50 chapters (reasonable range)            if 3 <= chapter_count <= 50 and chapter_count > best_count:                best_count = chapter_count                best_split = (pattern, splits)                # If we found a good split, use it        if best_split:            pattern, chapters = best_split            result = []                        # Skip first element (pre-chapter content) and process in groups of 3            for i in range(1, len(chapters), 3):                if i + 2 <= len(chapters):                    chapter_prefix = chapters[i].strip()                    chapter_number = chapters[i+1].strip()                    chapter_content = chapters[i+2].strip() if i+2 < len(chapters) else ""                                        # Create chapter title                    if chapter_prefix:                        chapter_title = f"{chapter_prefix} {chapter_number}"                    else:                        chapter_title = f"Chapter {chapter_number}"                                        if chapter_content:  # Only add if there's content                        result.append({                            'title': chapter_title,                            'content': chapter_content                        })                        return result if result else [{'title': 'Full Text', 'content': text}]                # If no pattern worked, try manual line-by-line analysis        return self._fallback_chapter_split(text)        def _fallback_chapter_split(self, text):        """        Fallback method: Analyze text line-by-line to find chapter markers        """        lines = text.split('\n')        chapters = []        current_chapter = None        current_content = []                for i, line in enumerate(lines):            line_stripped = line.strip()            line_upper = line_stripped.upper()                        # Check if this line looks like a chapter header            is_chapter = False            chapter_title = None                        # Very short line with numbers or roman numerals            if len(line_stripped) <= 30:                # Check for "CHAPTER X" format                if 'CHAPTER' in line_upper:                    is_chapter = True                    chapter_title = line_stripped                # Check for standalone numbers/roman numerals                elif re.match(r'^[IVXLCDM]+\.?$', line_upper):                    is_chapter = True                    chapter_title = f"Chapter {line_stripped}"                elif re.match(r'^\d+\.?$', line_stripped):                    is_chapter = True                    chapter_title = f"Chapter {line_stripped}"                # Check for "BOOK X", "PART X"                elif any(word in line_upper for word in ['BOOK', 'PART']) and len(line_stripped.split()) <= 3:                    is_chapter = True                    chapter_title = line_stripped                        if is_chapter and chapter_title:                # Save previous chapter if exists                if current_chapter and current_content:                    chapters.append({                        'title': current_chapter,                        'content': '\n'.join(current_content).strip()                    })                                # Start new chapter                current_chapter = chapter_title                current_content = []            else:                # Add to current chapter content                if line_stripped:  # Don't add empty lines at start                    current_content.append(line)                # Add last chapter        if current_chapter and current_content:            chapters.append({                'title': current_chapter,                'content': '\n'.join(current_content).strip()            })                # If we found chapters, return them; otherwise return full text        if len(chapters) >= 2:  # At least 2 chapters to be valid            return chapters        else:            return [{'title': 'Full Text', 'content': text}]# 사용 예시preprocessor = TextPreprocessor()print("✓ Text Preprocessor initialized")

In [ ]:
# 샘플 도서에 전처리 적용
if book_text:
    # 헤더/푸터 제거
    cleaned_text = preprocessor.remove_gutenberg_header_footer(book_text)
    cleaned_text = preprocessor.clean_text(cleaned_text)
    
    print(f"✓ Original length: {len(book_text)} characters")
    print(f"✓ Cleaned length: {len(cleaned_text)} characters")
    print(f"✓ Removed: {len(book_text) - len(cleaned_text)} characters")
    
    # 챕터 분할
    chapters = preprocessor.split_into_chapters(cleaned_text)
    print(f"\n✓ Found {len(chapters)} chapters")
    
    if chapters:
        print(f"\nFirst chapter: {chapters[0]['title']}")
        print(f"Content preview: {chapters[0]['content'][:300]}...")

### 3.2 핵심 요소 추출 (Entity Extraction)

인물, 장소, 시간 등 핵심 요소를 추출하여 스크립트 변환에 활용

In [ ]:
class EntityExtractor:
    """Named Entity Recognition을 통한 핵심 요소 추출 클래스"""
    
    def __init__(self):
        self.nlp = nlp_en
    
    def extract_entities(self, text, max_length=1000000):
        """
        텍스트에서 개체명 추출
        
        Args:
            text (str): 분석할 텍스트
            max_length (int): 처리할 최대 텍스트 길이
        
        Returns:
            dict: 카테고리별 개체명 사전
        """
        # SpaCy의 max_length 설정
        self.nlp.max_length = max_length
        
        # 텍스트 분석
        doc = self.nlp(text[:max_length])
        
        # 개체명 분류
        entities = {
            'PERSON': [],      # 인물
            'GPE': [],          # 지정학적 개체 (도시, 국가 등)
            'LOC': [],          # 위치
            'DATE': [],         # 날짜
            'TIME': [],         # 시간
            'ORG': [],          # 조직
            'EVENT': []         # 이벤트
        }
        
        for ent in doc.ents:
            if ent.label_ in entities:
                entities[ent.label_].append(ent.text)
        
        # 중복 제거 및 빈도수 계산
        for key in entities:
            entity_counts = defaultdict(int)
            for entity in entities[key]:
                entity_counts[entity] += 1
            entities[key] = sorted(entity_counts.items(), key=lambda x: x[1], reverse=True)
        
        return entities
    
    def get_main_characters(self, entities, top_n=10):
        """
        주요 인물 추출
        
        Args:
            entities (dict): extract_entities의 결과
            top_n (int): 반환할 주요 인물 수
        
        Returns:
            list: 주요 인물 리스트
        """
        return entities['PERSON'][:top_n]
    
    def get_main_locations(self, entities, top_n=10):
        """
        주요 장소 추출
        
        Args:
            entities (dict): extract_entities의 결과
            top_n (int): 반환할 주요 장소 수
        
        Returns:
            list: 주요 장소 리스트
        """
        locations = entities['GPE'] + entities['LOC']
        # 빈도수로 재정렬
        location_dict = defaultdict(int)
        for loc, count in locations:
            location_dict[loc] += count
        return sorted(location_dict.items(), key=lambda x: x[1], reverse=True)[:top_n]

# 사용 예시
extractor = EntityExtractor()
print("✓ Entity Extractor initialized")

In [ ]:
# 샘플 챕터에서 개체명 추출
if chapters:
    # 첫 번째 챕터 분석
    sample_chapter = chapters[0]['content']
    entities = extractor.extract_entities(sample_chapter)
    
    print("✓ Entity extraction completed\n")
    
    # 주요 인물
    main_characters = extractor.get_main_characters(entities, top_n=5)
    print("Main Characters:")
    for char, count in main_characters:
        print(f"  - {char}: {count} mentions")
    
    # 주요 장소
    main_locations = extractor.get_main_locations(entities, top_n=5)
    print("\nMain Locations:")
    for loc, count in main_locations:
        print(f"  - {loc}: {count} mentions")
    
    # 시간 정보
    if entities['DATE']:
        print("\nTemporal Information:")
        for date, count in entities['DATE'][:5]:
            print(f"  - {date}: {count} mentions")

### 3.3 스크립트 변환을 위한 데이터 구조화

도서 텍스트를 스크립트 학습에 적합한 형태로 변환

In [ ]:
class ScriptFormatter:
    """도서 텍스트를 스크립트 형식으로 변환하는 클래스"""
    
    def __init__(self):
        self.nlp = nlp_en
    
    def extract_dialogues(self, text):
        """
        텍스트에서 대화문 추출
        
        Args:
            text (str): 분석할 텍스트
        
        Returns:
            list: 대화문 리스트
        """
        # 따옴표로 둘러싸인 대화문 추출
        dialogue_pattern = r'["\']([^"\']+)["\']'
        dialogues = re.findall(dialogue_pattern, text)
        
        # 짧은 대화 필터링 (3단어 이상)
        dialogues = [d for d in dialogues if len(d.split()) >= 3]
        
        return dialogues
    
    def extract_narrative(self, text):
        """
        서술 부분 추출 (대화가 아닌 부분)
        
        Args:
            text (str): 분석할 텍스트
        
        Returns:
            str: 서술 텍스트
        """
        # 대화문 제거
        narrative = re.sub(r'["\'][^"\']+["\']', '', text)
        
        # 정제
        narrative = re.sub(r' +', ' ', narrative)
        narrative = re.sub(r'\n\s*\n', '\n\n', narrative)
        
        return narrative.strip()
    
    def create_scene_structure(self, chapter_text, entities):
        """
        챕터를 씬 구조로 변환
        
        Args:
            chapter_text (str): 챕터 텍스트
            entities (dict): 추출된 개체명
        
        Returns:
            dict: 씬 구조 정보
        """
        # 문장 단위로 분할
        doc = self.nlp(chapter_text[:100000])  # 처리 속도를 위해 제한
        sentences = [sent.text for sent in doc.sents]
        
        # 대화문과 서술 분리
        dialogues = self.extract_dialogues(chapter_text)
        narrative = self.extract_narrative(chapter_text)
        
        scene_data = {
            'characters': [char for char, _ in entities.get('PERSON', [])[:5]],
            'locations': [loc for loc, _ in (entities.get('GPE', []) + entities.get('LOC', []))[:3]],
            'dialogues': dialogues[:10],
            'narrative_sentences': sentences[:20],
            'total_sentences': len(sentences),
            'total_dialogues': len(dialogues)
        }
        
        return scene_data

# 사용 예시
formatter = ScriptFormatter()
print("✓ Script Formatter initialized")

In [ ]:
# 샘플 챕터를 씬 구조로 변환
if chapters and entities:
    scene_data = formatter.create_scene_structure(chapters[0]['content'], entities)
    
    print("✓ Scene structure created\n")
    print(f"Scene Information:")
    print(f"  - Main Characters: {', '.join(scene_data['characters'])}")
    print(f"  - Locations: {', '.join(scene_data['locations'])}")
    print(f"  - Total Sentences: {scene_data['total_sentences']}")
    print(f"  - Total Dialogues: {scene_data['total_dialogues']}")
    
    print("\nSample Dialogues:")
    for i, dialogue in enumerate(scene_data['dialogues'][:3], 1):
        print(f"  {i}. \"{dialogue}\"")
    
    print("\nSample Narrative:")
    for i, sentence in enumerate(scene_data['narrative_sentences'][:3], 1):
        print(f"  {i}. {sentence}")

## 4. 학습 데이터셋 구축

### 4.1 전체 파이프라인 실행

In [ ]:
class BookToScriptPipeline:
    """도서에서 스크립트 학습 데이터까지의 전체 파이프라인"""
    
    def __init__(self):
        self.collector = GutenbergCollector()
        self.preprocessor = TextPreprocessor()
        self.extractor = EntityExtractor()
        self.formatter = ScriptFormatter()
        
    def process_book(self, book_id):
        """
        단일 도서 처리
        
        Args:
            book_id (int): 도서 ID
        
        Returns:
            dict: 처리된 데이터
        """
        print(f"\nProcessing book {book_id}...")
        
        # 1. 도서 다운로드
        book_text = self.collector.download_book(book_id)
        if not book_text:
            return None
        print(f"  ✓ Downloaded ({len(book_text)} chars)")
        
        # 2. 전처리
        cleaned_text = self.preprocessor.remove_gutenberg_header_footer(book_text)
        cleaned_text = self.preprocessor.clean_text(cleaned_text)
        print(f"  ✓ Cleaned ({len(cleaned_text)} chars)")
        
        # 3. 챕터 분할
        chapters = self.preprocessor.split_into_chapters(cleaned_text)
        print(f"  ✓ Split into {len(chapters)} chapters")
        
        # 4. 각 챕터별 처리
        processed_chapters = []
        for i, chapter in enumerate(chapters[:5]):  # 처음 5개 챕터만 처리 (예시)
            # 개체명 추출
            entities = self.extractor.extract_entities(chapter['content'])
            
            # 씬 구조 생성
            scene_data = self.formatter.create_scene_structure(chapter['content'], entities)
            
            processed_chapters.append({
                'chapter_title': chapter['title'],
                'chapter_number': i + 1,
                'original_text': chapter['content'],
                'entities': entities,
                'scene_data': scene_data
            })
        
        print(f"  ✓ Processed {len(processed_chapters)} chapters")
        
        return {
            'book_id': book_id,
            'total_length': len(cleaned_text),
            'total_chapters': len(chapters),
            'processed_chapters': processed_chapters
        }
    
    def process_multiple_books(self, book_ids, output_dir='./processed_data'):
        """
        여러 도서 처리 및 저장
        
        Args:
            book_ids (list): 도서 ID 리스트
            output_dir (str): 출력 디렉토리
        
        Returns:
            list: 처리 결과 리스트
        """
        os.makedirs(output_dir, exist_ok=True)
        results = []
        
        for book_id in book_ids:
            result = self.process_book(book_id)
            if result:
                results.append(result)
                
                # JSON으로 저장
                output_file = os.path.join(output_dir, f'book_{book_id}.json')
                with open(output_file, 'w', encoding='utf-8') as f:
                    json.dump(result, f, ensure_ascii=False, indent=2)
                print(f"  ✓ Saved to {output_file}")
        
        print(f"\n✓ Completed processing {len(results)} books")
        return results

# 파이프라인 초기화
pipeline = BookToScriptPipeline()
print("✓ Pipeline initialized")

In [ ]:
# 샘플 도서 처리
sample_books = [1342]  # Pride and Prejudice
results = pipeline.process_multiple_books(sample_books, output_dir='./processed_data')

print("\n=== Processing Summary ===")
for result in results:
    print(f"\nBook ID: {result['book_id']}")
    print(f"  Total Length: {result['total_length']:,} characters")
    print(f"  Total Chapters: {result['total_chapters']}")
    print(f"  Processed Chapters: {len(result['processed_chapters'])}")

## 5. 성능 평가 지표 준비

### 5.1 BLEU Score 구현

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu, SmoothingFunction

class EvaluationMetrics:
    """모델 성능 평가를 위한 메트릭 클래스"""
    
    def __init__(self):
        self.smooth = SmoothingFunction()
    
    def calculate_bleu(self, reference, candidate, weights=(0.25, 0.25, 0.25, 0.25)):
        """
        BLEU 점수 계산
        
        Args:
            reference (str): 참조 텍스트
            candidate (str): 생성된 텍스트
            weights (tuple): n-gram 가중치
        
        Returns:
            float: BLEU 점수
        """
        # 토큰화
        reference_tokens = reference.split()
        candidate_tokens = candidate.split()
        
        # BLEU 계산 (smoothing 적용)
        bleu_score = sentence_bleu(
            [reference_tokens],
            candidate_tokens,
            weights=weights,
            smoothing_function=self.smooth.method1
        )
        
        return bleu_score
    
    def calculate_bleu_variants(self, reference, candidate):
        """
        다양한 BLEU 변형 계산 (BLEU-1 ~ BLEU-4)
        
        Args:
            reference (str): 참조 텍스트
            candidate (str): 생성된 텍스트
        
        Returns:
            dict: BLEU 변형 점수
        """
        return {
            'BLEU-1': self.calculate_bleu(reference, candidate, weights=(1, 0, 0, 0)),
            'BLEU-2': self.calculate_bleu(reference, candidate, weights=(0.5, 0.5, 0, 0)),
            'BLEU-3': self.calculate_bleu(reference, candidate, weights=(0.33, 0.33, 0.33, 0)),
            'BLEU-4': self.calculate_bleu(reference, candidate, weights=(0.25, 0.25, 0.25, 0.25))
        }

# 평가 메트릭 초기화
metrics = EvaluationMetrics()
print("✓ Evaluation Metrics initialized")

# 테스트 예시
reference_text = "The quick brown fox jumps over the lazy dog"
candidate_text = "The quick brown fox jumps over a lazy dog"

bleu_scores = metrics.calculate_bleu_variants(reference_text, candidate_text)
print("\nBLEU Score Examples:")
for metric, score in bleu_scores.items():
    print(f"  {metric}: {score:.4f}")

## 6. 다음 단계 및 계획

### 완료된 작업
- ✅ Project Gutenberg 데이터 수집 모듈
- ✅ 텍스트 전처리 (노이즈 제거, 정제)
- ✅ 챕터 분할 기능
- ✅ 개체명 추출 (인물, 장소, 시간)
- ✅ 스크립트 변환을 위한 데이터 구조화
- ✅ BLEU 평가 지표 구현

### 향후 작업 (10월 27일까지)
1. **데이터 수집 확대**
   - 더 많은 도서 다운로드 및 처리
   - 다양한 장르 확보 (소설, 드라마, 미스터리 등)
   - 국내 디지털 도서관 데이터 수집 방법 연구

2. **전처리 고도화**
   - 대화문과 서술 분리 정확도 향상
   - 장면 전환 감지 알고리즘 개발
   - 감정/톤 분석 추가

3. **데이터셋 품질 검증**
   - 결측치 및 이상치 검사
   - 데이터 통계 분석
   - 샘플 데이터 검토

### 모델링 준비 (10월 29일 이후)
- LLM 모델 선택 (GPT-2, T5, BART 등)
- Fine-tuning 전략 수립
- 학습 데이터 포맷 정의

### 성능 평가 계획
- BLEU Score: 텍스트 유사도 측정
- FVD (선택): 비디오 품질 평가 (여유가 있을 경우)
- CLIPScore (선택): 텍스트-이미지 유사도 (여유가 있을 경우)

## 7. 진행 상황 저장

In [ ]:
# 진행 상황 요약 저장
progress_summary = {
    'date': '2025-10-12',
    'phase': 'Data Preprocessing',
    'completed_tasks': [
        'Data collection module (Project Gutenberg)',
        'Text cleaning and preprocessing',
        'Chapter segmentation',
        'Entity extraction (characters, locations, time)',
        'Script formatting structure',
        'BLEU evaluation metric'
    ],
    'next_tasks': [
        'Expand dataset collection',
        'Enhance preprocessing accuracy',
        'Data quality validation',
        'Prepare for modeling phase'
    ],
    'deadline': '2025-10-27',
    'team_notes': 'Following professor\'s guidance - parallel work on dataset collection and framework setup'
}

# JSON으로 저장
with open('progress_summary.json', 'w', encoding='utf-8') as f:
    json.dump(progress_summary, f, ensure_ascii=False, indent=2)

print("✓ Progress summary saved to 'progress_summary.json'")
print("\nCurrent Status:")
print(f"  Phase: {progress_summary['phase']}")
print(f"  Deadline: {progress_summary['deadline']}")
print(f"  Completed: {len(progress_summary['completed_tasks'])} tasks")
print(f"  Remaining: {len(progress_summary['next_tasks'])} tasks")

### 3.1.1 챕터 분할 예시 및 디버깅

개선된 챕터 분할 알고리즘 테스트

In [ ]:
# 챕터 분할 테스트 및 디버깅
if 'book_text' in dir() and 'cleaned_text' in dir():
    print("=== 챕터 분할 디버깅 ===")
    print(f"\n1. 원본 텍스트 샘플 (처음 1000자):")
    print(cleaned_text[:1000])
    print("\n" + "="*60 + "\n")
    
    # 챕터 패턴 수동 검색
    print("2. 챕터 패턴 검색 결과:")
    
    patterns_to_test = [
        ("CHAPTER/Chapter + 번호", r'\n\s*(CHAPTER|Chapter)\s+([IVXLCDM]+|\d+)'),
        ("로마자/숫자만", r'\n\s*([IVXLCDM]+|\d+)\.?\s*\n'),
        ("BOOK/PART + 번호", r'\n\s*(BOOK|Book|PART|Part)\s+([IVXLCDM]+|\d+)'),
    ]
    
    for pattern_name, pattern in patterns_to_test:
        matches = re.findall(pattern, cleaned_text)
        print(f"  - {pattern_name}: {len(matches)}개 발견")
        if matches and len(matches) <= 10:
            print(f"    예시: {matches[:5]}")
    
    print("\n" + "="*60 + "\n")
    
    # 실제 챕터 분할 수행
    print("3. 챕터 분할 결과:")
    chapters = preprocessor.split_into_chapters(cleaned_text)
    print(f"  총 {len(chapters)}개 챕터 발견\n")
    
    # 처음 3개 챕터 미리보기
    for i, chapter in enumerate(chapters[:3], 1):
        print(f"  Chapter {i}:")
        print(f"    제목: '{chapter['title']}'")
        print(f"    길이: {len(chapter['content'])} 문자")
        print(f"    내용 미리보기: {chapter['content'][:150]}...")
        print()
    
    # 챕터 길이 통계
    if len(chapters) > 1:
        chapter_lengths = [len(ch['content']) for ch in chapters]
        print(f"\n4. 챕터 길이 통계:")
        print(f"  - 평균: {sum(chapter_lengths)/len(chapter_lengths):.0f} 문자")
        print(f"  - 최소: {min(chapter_lengths)} 문자")
        print(f"  - 최대: {max(chapter_lengths)} 문자")
    
    print("\n" + "="*60)
    print("✓ 챕터 분할 분석 완료")
else:
    print("먼저 샘플 도서를 다운로드하고 정제하세요.")